# DATA EXPLORATION

## 1. SELECT & SORT DATA

1.1. What is the `name` of the category with the highest `category_id` in the `dvd_rentals.category` table?

```sql
SELECT
  category_id,
  name
FROM dvd_rentals.category
ORDER BY category DESC
LIMIT 1;
```

1.2. For the films with the longest `length`, what is the `title` of the “R” rated film with the lowest `replacement_cost` in `dvd_rentals.film table`?

```sql
SELECT
  title,
  length,
  replacement_cost,
  rating
FROM dvd_rentals.film
ORDER BY
  length DESC,
  replacement_cost ASC
LIMIT 10;
```

1.3. Who was the `manager` of the store with the highest `total_sales` in the `dvd_rentals.sales_by_store` table?

```sql
SELECT
  manager,
  total_sales
FROM dvd_rentals.sales_by_store
ORDER BY total_sales DESC
LIMIT 1;
```

1.4. What is the `postal_code` of the city with the 5th highest `city_id` in the `dvd_rentals.address` table?

```sql
SELECT
  city_id,
  postal_code
FROM dvd_rentals.address
ORDER BY city_id DESC
LIMIT 5;
```

## 2. RECORD COUNTS & DISTINCT VALUES

2.1. Which `actor_id` has the most number of unique `film_id` records in the `dvd_rentals.film_actor` table?

```sql
SELECT
    actor_id,
    COUNT(DISTINCT film_id) AS number_of_unique_film
FROM dvd_rentals.film_actor
GROUP BY actor_id
ORDER BY number_of_unique_film DESC
LIMIT 1;
```

| actor_id | number_of_unique_film |
|----------|-----------------------|
| 107      | 42                    |

2.2 How many distinct `fid` values are there for the 3rd most common `price` value in the `dvd_rentals.nicer_but_slower_film_list` table?

```sql
SELECT
    price,
    COUNT(DISTINCT fid) AS number_of_fid
FROM dvd_rentals.nicer_but_slower_film_list
GROUP BY price
ORDER BY number_of_fid DESC
LIMIT 3;
```

| price | number_of_fid |
|-------|---------------|
| 0.99  | 340           |
| 4.99  | 334           |
| 2.99  | 323           |

2.3. How many unique `country_id` values exist in the `dvd_rentals.city` table? 

```sql
SELECT COUNT(DISTINCT country_id)
FROM dvd_rentals.city;
```

| count |
|-------|
| 109   |

2.4. What percentage of overall `total_sales` does the `Sports` category make up in the `dvd_rentals.sales_by_film_category` table?

```sql
SELECT
    category, 
    ROUND(100 * total_sales::NUMERIC / SUM(total_sales) OVER (), 2) AS percentage
FROM dvd_rentals.sales_by_film_category;
```

| category    | percentage |
|-------------|------------|
| Sports      | 7.88       |
| Sci-Fi      | 7.06       |
| Animation   | 6.91       |
| Drama       | 6.80       |
| Comedy      | 6.50       |
| Action      | 6.49       |
| New         | 6.46       |
| Games       | 6.35       |
| Foreign     | 6.33       |
| Family      | 6.28       |
| Documentary | 6.26       |
| Horror      | 5.52       |
| Children    | 5.42       |
| Classics    | 5.40       |
| Travel      | 5.27       |
| Music       | 5.07       |

2.5. What percentage of unique `fid` values are in the `Children` category in the `dvd_rentals.film_list` table?

```sql
SELECT
    category
,   COUNT(DISTINCT fid) AS frequency
,   ROUND(
        100 * COUNT(DISTINCT fid)::NUMERIC / SUM(COUNT(DISTINCT fid)) OVER (), 2
        )
        AS percentage
FROM dvd_rentals.film_list
GROUP BY category;
```

| category    | percentage |
|-------------|------------|
| Action      | 6.42       |
| Animation   | 6.62       |
| Children    | 6.02       |
| Classics    | 5.72       |
| Comedy      | 5.82       |
| Documentary | 6.82       |
| Drama       | 6.12       |
| Family      | 6.92       |
| Foreign     | 7.32       |
| Games       | 6.12       |
| Horror      | 5.62       |
| Music       | 5.12       |
| New         | 6.32       |
| Sci-Fi      | 6.12       |
| Sports      | 7.32       |
| Travel      | 5.62       |

## 3. IDENTIFYING DUPLICATE DATA

3.1. Which `id` value has the most number of duplicate records in the `health.user_logs` table?

```sql
WITH groupby_counts AS (
  SELECT
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic,
    COUNT(*) AS frequency
  FROM health.user_logs
  GROUP BY
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic
  HAVING COUNT(*) > 1
)
SELECT
  id,
  SUM(frequency) AS total_frequency
FROM groupby_counts
GROUP BY id
ORDER BY total_frequency DESC
LIMIT 1;
```

| id                                       | total_frequency |
|------------------------------------------|-----------------|
| 054250c692e07a9fa9e62e345231df4b54ff435d | 17279           |


3.2. Which `log_date` value had the most duplicate records after removing the max duplicate `id` value from question 1?

```sql
WITH groupby_counts AS (
  SELECT
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic,
    COUNT(*) AS frequency
  FROM health.user_logs
  WHERE id != '054250c692e07a9fa9e62e345231df4b54ff435d'
  GROUP BY
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic
  HAVING COUNT(*) > 1
)
SELECT
  log_date,
  SUM(frequency) AS total_frequency
FROM groupby_counts
GROUP BY log_date
ORDER BY total_frequency DESC
LIMIT 1;
```

| log_date    | total_frequency |
|-------------|-----------------|
| 2019-12-11  | 55              |

3.3. Which `measure_value` had the most occurences in the `health.user_logs` value when `measure = 'weight'`?

```sql
SELECT
  measure_value,
  COUNT(*) AS frequency
FROM health.user_logs
WHERE measure = 'weight'
GROUP BY measure_value
ORDER BY frequency DESC
LIMIT 1;
```

| measure_value | frequency |
|---------------|-----------|
| 68.49244787   | 109       |

3.4. How many single duplicated rows exist when `measure = 'blood_pressure'` in the `health.user_logs`? How about the total number of duplicate records in the same table?

```sql
WITH cte_duplicate AS (
  SELECT
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic,
    COUNT(*) AS frequency
  FROM health.user_logs
  WHERE measure = 'blood_pressure'
  GROUP BY
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic
)
SELECT
  COUNT(*) as single_duplicated_rows,
  SUM(frequency) as total_duplicated_records
FROM cte_duplicatevv
WHERE frequency > 1
LIMIT 1000;
```

| single_duplicated_rows | total_duplicated_records |
|---------------|-----------|
| 147 | 301 

3.5. What percentage of records `measure_value = 0` when `measure = 'blood_pressure'` in the `health.user_logs` table? How many records are there also for this same condition?

```sql
WITH cte_percentage AS (
  SELECT
    measure_value,
    COUNT(*) as frequency,
    SUM(COUNT(*)) OVER() as total_records
  FROM health.user_logs
  WHERE measure = 'blood_pressure'
  GROUP BY measure_value
)
SELECT
  measure_value,
  frequency,
  total_records,
  ROUND(100 * frequency::NUMERIC / total_records, 2) AS percentage
FROM cte_percentage
WHERE measure_value = 0;
```

| measure_value | frequency | total_records | percentage |
|-------------|-----------|---------------|------------|
| 0           | 562       | 2417          | 23.25      | 

3.6. What percentage of records are duplicates in the `health.user_logs table`?

```sql
WITH cte_distinct AS (
  SELECT DISTINCT *
  FROM health.user_logs
)
SELECT
  ROUND(
    100 * (
      (SELECT COUNT(*) FROM health.user_logs) -
      (SELECT COUNT(*) FROM cte_distinct)
    )::NUMERIC /
    (SELECT COUNT(*) FROM health.user_logs),
    2
  ) AS duplicate_percentage;
```

| duplicate_percentage |
|----------------------|
| 29.36                | 

## 4. SUMMARY STATISTICS

4.1. What is the average, median and mode values of blood glucose values to 2 decimal places?

```sql
SELECT
  ROUND(AVG(measure_value), 2) AS average_value,
  ROUND(CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY measure_value) AS NUMERIC), 2) AS median_value,
  ROUND(MODE() WITHIN GROUP (ORDER BY measure_value), 2) AS mode_value
FROM health.user_logs
WHERE measure = 'blood_glucose';
```

| average_value | median_value | mode_value |
|---------------|--------------|------------|
| 177.35        | 154          | 401        |

4.2 What is the most frequently occuring `measure_value` value for all blood glucose measurements?

```sql
SELECT
  measure_value,
  COUNT(*) as frequency
FROM health.user_logs
WHERE measure = 'blood_glucose'
GROUP BY measure_value
ORDER by frequency DESC
LIMIT 1;
```

| measure_value | frequency |
|---------------|-----------|
| 401           | 433       |

4.3. Calculate the 2 Pearson Coefficient of Skewness for blood glucose measures given the following formulas:

`Coefficient1 : ModeSkewness = (Mean − Mode) / StandardDeviation`
 
`Coefficient2 : MedianSkewness = 3 * (Mean − Median) / StandardDeviation`

```sql
WITH cte_blood_glucose_stats AS (
  SELECT
    AVG(measure_value) AS mean_value,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY measure_value) AS median_value,
    MODE() WITHIN GROUP (ORDER BY measure_value) AS mode_value,
    STDDEV(measure_value) AS stddev_value
  FROM health.user_logs
  WHERE measure = 'blood_glucose'
)
SELECT
  ( mean_value - mode_value ) / stddev_value AS pearson_corr_1,
  3 * ( mean_value - median_value ) / stddev_value AS pearson_corr_2
FROM cte_blood_glucose_stats;
```

| pearson_corr_1          | pearson_corr_2            |
|-------------------------|---------------------------|
| -0.19324890033272552535 | 0.06052397102651671       |